# Data Clean up
Authors: Abzer Kelminal (abzer.shah@uni-tuebingen.de) <br>
Edited by:  <br>
Input file format: .csv files or .txt files <br>
Outputs: .csv files  <br>
Dependencies: ggplot2, dplyr

This Notebook is used for cleaning the feature table, an output of metabolomics experiment, containing all the features with their corresponding intensities. The data cleanup steps involved are: 1) Blank removal 2) Imputation 3) Normlisation

# Brief introduction about the Test data:
- <p style='text-align: justify;'> Bacteria (B.subtilis and E.coli) were treated with a pool of antibiotics (Sulfamethoxazole, sulfadimethoxine, cyproconazole) and also a xenobiotic, Asulam, taken at a concentration lower than their MIC (minimum inhibitory concentration). </p>
- <p style='text-align: justify;'> The samples were collected at different timepoints, the compounds were extracted (with 50% EtOAc) and measured using LC-MS/MS. </p>
- <p style='text-align: justify;'> The goal of the experiment was to look for any potential biotransformation. eg: Drug or xenobiotic metabolism </p>

# Package installation:

In [ ]:
#installing and calling the necessary packages:
if (!require("ggplot2")) install.packages("ggplot2")
if (!require("dplyr")) install.packages("dplyr")

library(ggplot2)
library(dplyr)

# Getting the input files:

## 1) Reading the input data using URL (from GitHub):
Here, we can directly pull the data files from our Functional Metabolomics Github page:

In [ ]:
## Non-gap filled
nft_url <- 'https://raw.githubusercontent.com/Functional-Metabolomics-Lab/Metabolomics_SummerSchool_2022/main/data/MZmine/20220726_Xenobiotic_metabolism_Nongapfilled_quant.csv'
## Gap filled
ft_url <- 'https://raw.githubusercontent.com/Functional-Metabolomics-Lab/Metabolomics_SummerSchool_2022/main/data/MZmine/20220726_Xenobiotic_metabolism_Gapfilled_quant.csv'
md_url <- 'https://raw.githubusercontent.com/Functional-Metabolomics-Lab/Metabolomics_SummerSchool_2022/main/data/20220726_Xenobiotic_Metabolism_metadata.txt'

In [ ]:
nft <- read.csv(nft_url, header = T, check.names = F)
ft <- read.csv(ft_url, header = T, check.names = F)
md <- read.csv(md_url, header = T, check.names = F, sep = '\t')

## 2) Setting a local working directory and creating an automatic result directory:
<p style='text-align: justify;'> Works well with Jupyter Notebook. If you are working with Jupyter Notebook, you can simply copy the folder path of your input files from your local computer to the output line of the next cell. It will be set as your working directory (or working folder)</p> 
For ex: D:\User\Project\Test_Data <br>
<p style='text-align: justify;'> For Google Collab, we can upload the necessary files into a new folder using the 'Files' icon on the left and set the folder as working directory. And all the ouput files will be saved here as well and you need to download them finally into your local computer </p>

In [ ]:
# setting the current directory as the working directory
Directory <- normalizePath(readline("Enter the path of the folder with input files: "),"/",mustWork=FALSE)
setwd(Directory)

In [ ]:
getwd() #to get the working directory 

In [ ]:
# Getting all the files in the folder
dirs <- dir(path=paste(getwd(), sep=""), full.names=TRUE, recursive=TRUE)
folders <- unique(dirname(dirs))
files <- list.files(folders, full.names=TRUE)
files_1 <- basename((files))
files_2 <- dirname((files))
# Creating a Result folder
dir.create(path=paste(files_2[[1]], "_Results", sep=""), showWarnings = TRUE)
fName <-paste(files_2[[1]], "_Results", sep="")

print(files_1)

In the following line, enter the required file index numbers separated by commas. For example as: 1,2,3. The accepted file formats are csv,txt and tsv.

In [ ]:
input <- as.double(unlist(strsplit(readline("Specify the file index of gapfilled feature-file, metadata separated by commas:"), split=",")))

#Gets the extension of each file. Ex:csv
pattern <- c()
for (i in files_1){
  sep_file <- substr(i, nchar(i)-2,nchar(i))
  pattern <- rbind(pattern,sep_file)
}
#pattern

ft <- read.csv(files_1[input[1]],sep = ifelse(pattern[input[1]]!="csv","\t",","), header=TRUE,check.names = FALSE) # By applying 'row.names = 1', the 1st column 'ID' becomes the row names
md <-read.csv(files_1[input[2]], sep = ifelse(pattern[input[2]]!="csv","\t",","), header=TRUE,check.names = FALSE)

<p style='text-align: justify;'> Since non gapfilled feature file is not a direct output of mzMine, not everybody will have it. Therefore, we have the next cell separately. For the imputation step, we can also impute the feature table without a non-gapfilled feature file.</p>

 More about this in the later section: [Imputation with a Cutoff LOD](#Impute_LOD)

In [ ]:
#If you have non gapfilled feature file:
if(readline("Do you have non gap-filled feature table? Y/N:")=="Y"){
  x <- as.double(readline("Enter the ID number of non-gap-filled feature file:"))
  nft<- read.csv(files_1[x],sep=ifelse(pattern[x]!="csv","\t",","), header = TRUE,check.names = FALSE)
}

Lets check if the data has been read correclty!!

In [ ]:
head(ft)
dim(ft)

In [ ]:
head(nft)
dim(nft)

In [ ]:
head(md)
dim(md)

<p style='text-align: justify;'> In the next cell, we are trying to bring the feature table and metadata in the correct format such as the rownames of metadata and column names of feature table are the same. They both are the file names and they need to be the same, as from now on, we will call the columns in our feature table based on our metadata information. Thus, using the metadata, the user can filter their data easily. You can also directly deal with your feature table without metadata by getting your hands dirty with some coding!! But having a metadata improves the user-experience greatly. </p>

In [ ]:
#Removing Peak area extensions
colnames(ft) <- gsub(' Peak area','',colnames(ft))
colnames(nft) <- gsub(' Peak area','',colnames(nft))
md$filename<- gsub(' Peak area','',md$filename)

#Removing if any NA columns present in the md file
md <- md[,colSums(is.na(md))<nrow(md)]

#Changing the row names of the files
rownames(md) <- md$filename
md <- md[,-1]
rownames(ft) <- paste(ft$'row ID',round(ft$'row m/z',digits = 3),round(ft$'row retention time',digits = 3), sep = '_')
rownames(nft) <- paste(nft$'row ID',round(nft$'row m/z',digits = 3),round(nft$'row retention time',digits = 3), sep = '_')

#Picking only the files with column names containing 'mzML'
ft <- ft[,grep('mzML',colnames(ft))]
nft <- nft[,grep('mzML',colnames(nft))]

Lets check the files once again!!

In [ ]:
head(nft)
dim(nft)

In [ ]:
head(ft)
dim(ft)

In [ ]:
head(md)
dim(md)

# Splitting the data into Control and Samples using Metadata:
<a id="data_split"></a>

For the first step: Blank removal, we need to split the data as spectra obtained from control blanks and samples respectively using the metadata. More about Blank removal in the next section.

Let's call the metadata and feature table in different names so that we don't change the original files.

In [ ]:
Meta_Filter <- md
input_data <- ft

From the below cell output, we get an idea of the multiple levels in each of the metioned attributes in the metadata

In [ ]:
 lev <- c()
  for(i in 1:ncol(Meta_Filter)){
    x <- levels(as.factor(Meta_Filter[,i]))
    if(is.double(Meta_Filter[,i])==T){x=round(as.double(x),2)}
    x <-toString(x)
    lev <- rbind(lev,x)
  }

Info_mat <- data.frame(INDEX=c(1:ncol(Meta_Filter)),ATTRIBUTES=colnames(Meta_Filter),LEVELS=lev,row.names=NULL)
Info_mat

### Batch-wise splitting of the data:

In our test dataset, we have used 2 species data: B.sub and E.coli. If you want to split the data batch-wise, run the next cell. For example: We can split it as B.sub + Media and/or E.coli + Media

In [ ]:
#Splitting batch-wise:
Batch_info <- ifelse(readline("Do you want to split your data batch-wise? Y/N: ")=="Y",as.double(readline("Please enter the index of the attributes having batch information:")),"No")
if(is.numeric(Batch_info)==TRUE){
    Levels_Batch <- levels(as.factor(Meta_Filter[,Batch_info]))
    data.frame(ID=c(1:length(Levels_Batch)),Levels=Levels_Batch)
   
    Cdtn <-as.double(unlist(strsplit(readline("Enter the index number of condition(s) you want to KEEP (separated by commas):"), split=',')))
    
    #Getting all the rows of metadata that satisfies each element of the condition and storing it as an element in Split_data list
    Split_data <-list()
    for (j in 1:length(Cdtn)){
      Split_data[[j]] <- Meta_Filter[(Meta_Filter[,Batch_info]==Levels_Batch[Cdtn[j]]),]
    }
    
    Batch_data <-do.call(rbind, Split_data) # unlists the Split data and combines them by row
    flush.console()  
    Sys.sleep(0.2)
   
    head(Batch_data)  #Visualising the Batch_data
    dim(Batch_data)
    }

Once we subset the data according to a batch, we can further proceed to split the control blank from the sample in the below cell. If no batch data present, you can simply split your metadata into control blank and sample.

In [ ]:
#If batch data exists, it will take it as "data", else take Meta_filter as "data"
if(exists("Batch_data")==T){data <-Batch_data}else{data <-Meta_Filter}

Info_mat
Condition <- as.double(unlist(readline("Enter the index of the attribute to split sample and control:")))

Levels_Cdtn <- levels(as.factor(data[,Condition[1]]))
print(matrix(Levels_Cdtn,length(Levels_Cdtn)))
 
  #Among the shown levels of an attribute, select the ones to keep
Ctrl_id <- as.double(unlist(readline("Enter the index of your BLANK:")))
paste0('You chosen blank is:',Levels_Cdtn[Ctrl_id])
  
  #Splitting the data into control and samples based on the metadata
md_Ctrl <- data[(data[,Condition] == Levels_Cdtn[Ctrl_id]),]
Ctrl <- input_data[,which(colnames(input_data)%in%rownames(md_Ctrl))] 
md_Samples <- data[(data[,Condition] != Levels_Cdtn[Ctrl_id]),]
Samples <- input_data[,which(colnames(input_data)%in%rownames(md_Samples))] 

In [ ]:
head(Ctrl)
dim(Ctrl)

In [ ]:
head(Samples)
dim(Samples)

# Creating a function named FrequencyPlot:  
The below function takes the two input datatables: for example, gapfilled and non-gapfilled as its inputs, calculates the frequency distribution of the data in the order of 10 and produces a grouped barplot showing the distribution as output. The frequency plot shows where the features are present in higher number.

In [ ]:
#'Global' settings for plot size in the output cell
#options(repr.plot.width=10, repr.plot.height=8,res=600) #For google collab
options(repr.plot.width=5, repr.plot.height=3) #For Jupyter Notebook

In [ ]:
FrequencyPlot <- function(x1,x2){
  
   #creating bins from -1 to 10^10 using sequence function seq()
    bins <- c(-1,0,(1 * 10^(seq(0,10,1)))) 
    
    #cut function cuts the give table into its appropriate bins
    scores_x1 <- cut(as.matrix(x1),bins,labels = c('0','1','10','1E2','1E3','1E4','1E5','1E6','1E7','1E8','1E9','1E10')) 
    
    #transform function convert the tables into a column format: easy for visualization 
    Table_x1<-transform(table(scores_x1)) #contains 2 columns: "scores_x1", "Freq"
    
    #Repeating the same steps for x2
    scores_x2 <- cut(as.matrix(x2),bins,labels = c('0','1','10','1E2','1E3','1E4','1E5','1E6','1E7','1E8','1E9','1E10'))
    Table_x2<-transform(table(scores_x2))
  
    #Getting the names of x1 and x2
    arg1 <- deparse(substitute(x1))
    arg2 <-deparse(substitute(x2))
    
    #Creating a data frame for plotting
    data_plot <- as.data.frame(c(Table_x1$Freq,Table_x2$Freq)) #Concatenating the frequency info of both tables rowwise
    colnames(data_plot) <- "Freq" #naming the 1st column as 'Freq'
    data_plot$Condition <- c(rep(arg1,12),rep(arg2,12)) #adding a 2nd column 'Condition', which just repeats the name of x1 and x2 accordingly
    data_plot$Range_bins <- rep(Table_x1$scores_x1,2) #Adding 3rd column 'Range Bins'
    data_plot$Log_Freq <- log(data_plot$Freq+1) #Log scaling the frequency values
    
    ## GGPLOT2
    BarPlot <- ggplot(data_plot, aes(Range_bins, Log_Freq, fill = Condition)) + 
    geom_bar(stat="identity", position = "dodge", width=0.4) + 
    scale_fill_brewer(palette = "Set1") +
    ggtitle(label="Frequency plot") +
    xlab("Range") + ylab("(Log)Frequency") + labs(fill = "Data Type") + 
    theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1)) +   # setting the angle for the x label
    theme(axis.text.y = element_text(angle = 45, vjust = 0.5, hjust=1)) +   # setting the angle for the y label
    theme(plot.title = element_text(hjust = 0.5)) # centering the plot title
  
    print(BarPlot)
}  

<p style='text-align: justify;'> As you have noticed, the above cell didn't produce any outputs. Here, we created our own function called 'FrequencyPlot'. which works similar to "FREQUENCY" function in MS Excel. But we have also added codes to print a nice ggplot as ouput of the function. Hence, like 'print' function prints an output, whenever we use 'FrequencyPlot' function with the input variables, it will produce an output plot using the calculation in the above cell. </p>

**Now that we have our data ready, we can start with the cleanup steps!!**

# Step1: Blank Removal

<p style='text-align: justify;'> In LC-MS/MS, we use solvents called Blanks which are usually injected time-to-time to prevent carryover of the sample. The features coming from these Blanks would also be detected by LC-MS/MS instrument. Our goal here is to remove these features from our samples. The other blanks that can be removed are: Signals coming from growth media alone in terms of microbial growth experiment, signals from the solvent used for extraction methods and so on. Therefore, it is best practice to measure mass spectra of these blanks as well in addition to your sample spectra. </p>

**How do we remove these blank features?** </br> 
<p style='text-align: justify;'> Since we have the feature table split into Control (Blanks) and Sample groups now, we can compare control to the sample to identify the background features coming from control. A common filtering method is to use a cutoff to remove features that are not present sufficient enough in our biological samples. </p>

The steps followed in the next few cells are:
1. <p style='text-align: justify;'> We find an average for all the feature intensities in your control set and sample set. Therefore, for n no.of features in a control or sample set, we get n no.of averaged features. </p>
2. <p style='text-align: justify;'> Next, we get a ratio of this average_control vs average_sample. This ratio Control/sample tells us how much of that particular feature of a sample gets its contribution from control. If it is more than 30% (or Cutoff as 0.3), we consider the feature as noise. </p>
3. <p style='text-align: justify;'> The resultant information (if ratio > Cutoff or not) is stored in a bin such as 1 = Noise or background signal, 0 = Feature Signal</p>
4. <p style='text-align: justify;'> We count the no.of features in the bin that satisfies the condition ratio > cutoff, and consider those features as 'noise or background features' and remove them. </p>

For a dataset containing several batches, the filtering steps are performed batch-wise and it can be done using the previous section [Batch-wise splitting of the data](#data_split)

**<font color='red'> The Cutoff used to obtain the all the files in MZmine Results folder is 0.3 </font>**

In [ ]:
if(readline('Do you want to perform Blank Removal- Y/N:')=='Y'){
    
    #When cutoff is low, more noise (or background) detected; With higher cutoff, less background detected, thus more features observed
    Cutoff <- as.numeric(readline('Enter Cutoff value between 0.1 & 1:')) # (i.e. 10% - 100%). Ideal cutoff range: 0.1-0.3
    
    #Getting mean for every feature in Ctrl and Samples
    Avg_ctrl <- rowMeans(Ctrl, na.rm= FALSE, dims = 1) # set na.rm = FALSE to check if there are NA values. When set as TRUE, NA values are changed to 0
    Avg_samples <- rowMeans(Samples, na.rm= FALSE, dims = 1)
    
    #Getting the ratio of Ctrl vs Sample
    Ratio_Ctrl_Sample <- (Avg_ctrl+1)/(Avg_samples+1)
    
    # Creating a bin with 1s when the ratio>Cutoff, else put 0s
    Bg_bin <- ifelse(Ratio_Ctrl_Sample > Cutoff, 1, 0 )
    Blank_removal <- cbind(Samples,Bg_bin)

    # Checking if there are any NA values present. Having NA values in the 4 variables will affect the final dataset to be created
    temp_NA_Count <-cbind(Avg_ctrl ,Avg_samples,Ratio_Ctrl_Sample,Bg_bin)
    
    print('No of NA values in the following columns:')
    print(colSums(is.na(temp_NA_Count)))

     #Calculating the number of background features and features present
    print(paste("No.of Background or noise features:",sum(Bg_bin ==1,na.rm = TRUE)))
    print(paste("No.of features after excluding noise:",(nrow(Samples) - sum(Bg_bin ==1,na.rm = TRUE)))) 

    Blank_removal <- Blank_removal %>% filter(Bg_bin == 0) # Taking only the feature signals
    Blank_removal <- as.matrix(Blank_removal[,-ncol(Blank_removal)]) # removing the last column Bg_bin 
}

In [ ]:
head(Blank_removal)
dim(Blank_removal)

# Step 2: Imputation

<p style='text-align: justify;'> For several reasons, real world datasets might have some missing values in it, in the form of NA, NANs or 0s. Eventhough the gapfilling step of MZmine fills the missing values, we still end up with some missing values or 0s in our feature table. This could be problematic for statistical analysis. </p> 
<p style='text-align: justify;'> In order to have a better dataset, we cannot simply discard those rows or columns with missing values as we will lose a chunk of our valuable data. Instead we can try imputing those missing values. Imputation involves replacing the missing values in the data with a meaningful, reasonable guess. There are several methods, such as: </p> 
  
1) Mean imputation (replacing the missing values in a column with the mean or average of the column)  
2) Replacing it with the most frequent value  
3) Several other machine learning imputation methods such as k-nearest neighbors algorithm(k-NN), Hidden Markov Model(HMM)

## 1) Imputation using Non-gapfilled feature table:
One such method, we are going to use is: **to replace the zeros from the gapfilled quant table with the non-gap filled table** we get from MZmine. In order to do that, we can visualize our data distribution using the frequenct plot

In [ ]:
GapFilled <-Blank_removal
if(exists("nft")==T){NotGapFilled <-nft}

In [ ]:
if(readline('Do you want to perform Imputation with minimum value of NonGapFilled table? - Y/N:')=='Y'){
    plot<- FrequencyPlot(GapFilled,NotGapFilled)
    
    Arg1 = plot$data$Condition[1]
    Arg2 = plot$data$Condition[13]
    
    # getting the 2nd minimum value of non-gap filled data. (The first minimum value in the data table is usually zero)
    RawLOD <- round(min(NotGapFilled[NotGapFilled!=min(NotGapFilled)]))
    
    print(paste0("The minimum value greater than 0 for ",Arg1,":", round(min(GapFilled[GapFilled!=min(GapFilled)]))))
    print(paste0("The minimum value greater than 0 for ",Arg2,":", RawLOD))

    Imputed <- GapFilled
    Imputed[Imputed<RawLOD] <- RawLOD # Replacing values<RawLOD with RawLOD
    head(Imputed) 
}

In [ ]:
write.csv(Imputed, file.path(fName,paste0('Imputed_QuantTable_filled_with_MinValue_',RawLOD,'_CutOff_Used_',Cutoff,'.csv')),row.names =TRUE) 

## 2) Imputation with a Cutoff LOD:
<a id="Impute_LOD"></a>

Instead of replacing with the minimum value of nft, we can also only use ft and see the frquency distribution of its features with the frequency plot. The frequency plot shows where the features are present in higher number.

For ex: If until range 10-100, (shown in the figure as 1E2) there are no or very less features, we want to exclude until that range and consider from range (100-1000), or, in other words, '1E3' or '1000' as Cutoff_LOD. This value will be used to replace the zeros in the data table.

In [ ]:
FrequencyPlot(GapFilled,GapFilled)

In [ ]:
if(readline('Do you want to perform Imputation with a Cutoff LOD? - Y/N:')=='Y'){
    Cutoff_LOD <-as.numeric(readline("Enter your Cutoff LOD as seen in the plot:"))  #Enter the LOD value as seen in the frequency plot
    Imputed <- GapFilled
    Imputed[Imputed <Cutoff_LOD] <- Cutoff_LOD
    head(Imputed)
}

In [ ]:
dim(Imputed)

In [ ]:
write.csv(Imputed,file.path(fName, paste0('Imputed_QuantTable_filled_with_',Cutoff_LOD,'_CutOff_Used_',Cutoff,'_Bsub','.csv')),row.names =TRUE)

In [ ]:
#removing all the rows with only cutoff values:
#Imputed<-Imputed[rowMeans(Imputed)!= Cutoff_LOD,]  

# Step 3:Normalization
The following code performs sample-centric (column-wise) normalisation:

In [ ]:
if (readline("Do you want to perform Normalization: Y/N:") == 'Y'){
    #Getting column-wise sums of the input-data
    sample_sum <- colSums(Imputed, na.rm= TRUE, dims = 1)
    
    #Dividing each element of a particular column with its column sum
    Normalized_data <- c()
    for (i in 1:ncol(Imputed)){
        x <- Imputed[,i] / sample_sum[i]
        Normalized_data <- cbind(Normalized_data, x)
    }
    colnames(Normalized_data) <- names(sample_sum)
    
    head(Normalized_data)
} else return(head(Imputed))
  
print(paste('No.of NA values in Normalized data:',sum(is.na(Normalized_data)== TRUE)))

In [ ]:
dim(Normalized_data)

In [ ]:
write.csv(Normalized_data, file.path(fName,'Normalised_Quant_table_Bsub_OnlyCutOffsRemoved.csv'),row.names =TRUE) 